### CIFAR-10 Classification with Pytorch Pretrained RegNet

+ RegNet (Regularized Network) is a family of network architectures designed through a network design space paradigm. 

+ Instead of manually designing individual architectures, RegNet defines a set of design parameters and uses a simple, regularized model to sample efficient networks from this space.

+ In general terms the design focuses on finding simple design rules that would lead to model with good performance.

+ To do so 
  
    + it scales network width and depth in a systematic way, and
  
    + it uses a block-based structure with efficient bottleneck blocks.

+ The code in this notebook uses RegNet to do image classification of the CIFAR-10 Data.

+ RegNet was desined to target ImageNet images, which are typically 224x224 pixels.

+ CIFAR-10 images are 32x32 pixels. This means there is a significant difference in image size.

+ Directly feeding 32x32 images into RegNet will result in a mismatch in the spatial dimensions of the feature maps.

+ For CIFAR-10, the first conv layer of RegNet needs to be adepted for 32x32 images.

+ The last dense layer (classification head) of RegNet is designed for ImageNet's 1000 classes.

+ That layer must be replaced with a new dense layer to classify the 10 classes in CIFAR-10.
 



In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.models import regnet_y_400mf, RegNet_Y_400MF_Weights

In [2]:
# Load CIFAR-10 dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # ImageNet normalization
    transforms.Resize(224) # RegNet pretrained models expect 224x224 images
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # ImageNet normalization
    transforms.Resize(224)
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
# Load pretrained RegNet model
weights = RegNet_Y_400MF_Weights.DEFAULT
net = regnet_y_400mf(weights=weights)

# Freeze all layers except the final fully connected layer
for param in net.parameters():
    param.requires_grad = False

# Replace the final fully connected layer for CIFAR-10 (10 classes)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 10)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.fc.parameters(), lr=0.001) # Optimize only the new FC layer

In [ ]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_params(net)

print( net ) 

4410

In [ ]:
# Ensure that training loop is using the GPUs in the system
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 1.330
[2,   200] loss: 0.858
[3,   200] loss: 0.798
[4,   200] loss: 0.771
[5,   200] loss: 0.745
[6,   200] loss: 0.739
[7,   200] loss: 0.741
[8,   200] loss: 0.737
[9,   200] loss: 0.730
[10,   200] loss: 0.728
Finished Training


In [11]:
# Test the network on the test data
correct = 0
total = 0
with torch.no_grad():
    net.eval() #set to evaluation mode
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 77 %


In [10]:
# Test the network and get per-class accuracy
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    net.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(labels.size(0)): # Iterate through samples in the batch.
            label = labels[i].item()
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 77 %
Accuracy of   car : 85 %
Accuracy of  bird : 73 %
Accuracy of   cat : 58 %
Accuracy of  deer : 73 %
Accuracy of   dog : 76 %
Accuracy of  frog : 85 %
Accuracy of horse : 72 %
Accuracy of  ship : 83 %
Accuracy of truck : 88 %
